In [1]:
%%script echo skipping
%conda install jupyter pytorch
%pip install 'git+https://github.com/deepset-ai/haystack.git' sentence-transformers 'txtai[pipeline-data]' elasticsearch-haystack gradio

skipping


In [2]:
from haystack import Pipeline, Document
from elasticsearch_haystack.embedding_retriever import ElasticsearchEmbeddingRetriever
from elasticsearch_haystack.document_store import ElasticsearchDocumentStore
from haystack.components.embedders import SentenceTransformersTextEmbedder, SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from txtai.pipeline import Textractor
from haystack.dataclasses import ChatMessage
from haystack.components.writers.document_writer import DuplicatePolicy
from haystack.components.builders import DynamicChatPromptBuilder

from haystack.components.generators.chat import OpenAIChatGenerator

textract = Textractor(paragraphs=True)

document_store = ElasticsearchDocumentStore(hosts="http://localhost:9200", embedding_similarity_function="cosine")

document_embedder = SentenceTransformersDocumentEmbedder(
    model_name_or_path="BAAI/llm-embedder",
    prefix="Represent this document for retrieval: "
)
document_embedder.warm_up()

documents = []

In [3]:
%%script echo skipping
for paragraph in textract("zarathustra-critical-guide.html"):
    if len(paragraph) > 32:
        documents.append(
            Document(
                meta={'name': "Nietzsche's 'Thus Spoke Zarathustra': A Critical Guide"},
                content=paragraph
            )
        )

for paragraph in textract("zarathustra.md"):
    if len(paragraph) > 32:
        documents.append(
            Document(
                meta={'name': "Thus Spoke Zarathustra"},
                content=paragraph
            )
        )

skipping


In [4]:
document_writer = DocumentWriter(document_store = document_store)
indexing_pipeline = Pipeline()
indexing_pipeline.add_component(instance=document_embedder, name="embedder")
indexing_pipeline.add_component(instance=document_writer, name="writer")
indexing_pipeline.connect("embedder", "writer")
indexing_pipeline.draw("indexing_pipeline.png")


In [5]:
%%script echo skipping
indexing_pipeline.run(
    {
        "embedder": {
            "documents": documents
        },
        "writer": {
            "policy": DuplicatePolicy.OVERWRITE
        }
    }
)

skipping


In [6]:
retriever = ElasticsearchEmbeddingRetriever(
    document_store=document_store,
    top_k=10
)

text_embedder = SentenceTransformersTextEmbedder(
        model_name_or_path="BAAI/llm-embedder",
        prefix="Represent this query for retrieving relevant documents: "
    )

template = """
Given the following information, follow my instruction.

Context: 
{% for document in documents %}
    {{ document.content }}
{% endfor %}

My Instruction: {{ question }}
"""

prompt_builder = DynamicChatPromptBuilder(runtime_variables=["documents"])


In [7]:
llm = OpenAIChatGenerator(api_base_url="http://127.0.0.1:42424/v1", api_key="not-needed")
rag_pipeline = Pipeline()
rag_pipeline.add_component("text_embedder", text_embedder)
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("llm", llm)
rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")
#rag_pipeline.draw("rag_pipeline.png")

In [12]:


def send(message, history):
    messages = [ChatMessage.from_user(template)]
    response = rag_pipeline.run(
        {
            "text_embedder": {"text": message},
            "prompt_builder": {
                "template_variables": {"question": message},
                "prompt_source": messages
            }
        }
    )
    return response.get('llm').get('replies')[-1].content

#send("What is a superman?", None)

In [13]:
import gradio as gr

demo = gr.ChatInterface(fn=send, title="RAG2A")
demo.launch(inline=False)

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
